<a href="https://colab.research.google.com/github/rinalditoryanari/ModelSleepApneaEEG/blob/full-original/EEG_Sleep_Apnea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Library

In [ ]:
from google.colab import drive
import tensorflow as tf
import os
import numpy as np
import random
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
print(tf. __version__) 

2.12.0


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


###EDF REader


In [ ]:
!pip install pyedflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.2 MB/s eta 0:00:00


In [ ]:
import pyedflib

# TRY 1
Jadi ini tuh rencananya, data txt itu bakal dipecah 1 row di txt untuk 1 file text
Udah termasuk ngapus header nya

## Directory Drive

### Multiple Files txt

In [ ]:
## dataset 61 sleep edf
vector_dir = "/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/"

In [ ]:
## dataset physonet
vector_dir = ""

In [ ]:
## Ambil list directory di vector
eeg_vec_dir = tf.io.gfile.glob(str(vector_dir) +'*.txt')
print(eeg_vec_dir)
eeg_vec_dir = tf.random.shuffle(eeg_vec_dir)
print("files : ", len(eeg_vec_dir))

['/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7132.txt', '/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7141.txt', '/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7071.txt', '/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7092.txt', '/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7201.txt', '/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7151.txt']
files :  6


### Single File

In [ ]:
vector_dir = "/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7071.txt"

In [ ]:
## Ambil list directory di vector
eeg_vec_dir = [vector_dir]
print(eeg_vec_dir)
eeg_vec_dir = tf.random.shuffle(eeg_vec_dir)
print("files : ", len(eeg_vec_dir))

['/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7071.txt']
files :  1


### eeg cap File

In [ ]:
sample_dir = "/content/drive/MyDrive/dataset/eegcap"
labels = ["normal", "sdb"]

## Bikin Directory Sample

In [ ]:
# Define a function to create the directories for each label
def makeDirectory(labels, sample_dir):
  for label in labels:
    os.makedirs(sample_dir+'/'+label, exist_ok=True)
    print("Created directory for", label)

##Extract EEG into individual file

In [ ]:
# Define a function to split each EEG file into individual recordings
def pecahSatu(filepath, labels, sample_dir):
  with open(filepath, 'r') as file:
    rows = file.readlines()

  num_rows_per_recording = 1
  num_recordings = len(rows) // num_rows_per_recording

  # Save each recording to a separate file
  for i in range(num_recordings):
      start_index = i * num_rows_per_recording
      end_index = start_index + num_rows_per_recording
      recording = rows[start_index:end_index]

      text = recording[0].split('\t')
      title = text[0]
      recording = text[6:6006]
      
      delimiter = ' '
      recording_str = delimiter.join(recording)

      # Choose a random label
      label = random.choice(labels)

      # Save the recording to a file
      filename = f'{label}_{title}.txt'
      filepath = os.path.join(sample_dir, label, filename)
      with open(filepath, 'w') as file:
          file.writelines(recording_str)

      print("Saved recording to", filepath)

## Eksekusi Extract

### Multiple Files

In [ ]:
import random

eeg_vec_dir = tf.io.gfile.glob(str(vector_dir) +'*.txt')
sample_dir = "/content/drive/MyDrive/dataset/sample/"


for dir in eeg_vec_dir:
  words = ["sa", "normal"]

  makeDirectory(words)
  pecahSatu(dir, words)

sa
normal
normal	sa
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_30.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_60.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/sa/7132_90.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/sa/7132_120.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_150.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/sa/7132_180.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_210.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_240.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_270.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/sa/7132_300.txt' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='drive/MyDrive/sample/normal/7132_3

KeyboardInterrupt: ignored

### Single Files

In [ ]:
import random

sample_dir = "/content/drive/MyDrive/dataset/sample"
labels = ["sa", "normal"]

In [ ]:
# convert the EagerTensor object to a NumPy array
eeg_vec_dir = np.array(eeg_vec_dir)

# convert the array to a Python list
eeg_vec_dir = eeg_vec_dir.tolist()

for dir in eeg_vec_dir:

  print(dir)

  makeDirectory(labels, sample_dir)
  pecahSatu(dir, labels, sample_dir)

b'/content/drive/MyDrive/dataset/eeg/61-sleep-edf/Vector/Vector_7071.txt'
Created directory for sa
Created directory for normal
Saved recording to /content/drive/MyDrive/dataset/sample/sa/sa_7071_30.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_60.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_90.txt
Saved recording to /content/drive/MyDrive/dataset/sample/sa/sa_7071_120.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_150.txt
Saved recording to /content/drive/MyDrive/dataset/sample/sa/sa_7071_180.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_210.txt
Saved recording to /content/drive/MyDrive/dataset/sample/sa/sa_7071_240.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_270.txt
Saved recording to /content/drive/MyDrive/dataset/sample/normal/normal_7071_300.txt
Saved recording to /content/drive/MyDrive/dataset/sample/sa/sa_7071

##READ INDIVIDUAL

### CAP (edf)

In [ ]:
import pyedflib

# open the EDF file
edf_file = pyedflib.EdfReader('/content/drive/MyDrive/dataset/eegcap/normal/n1.edf')
print(edf_file)

# get the channel names and sample rates
channel_names = edf_file.getSignalLabels()
print(channel_names)

sample_rates = edf_file.getSampleFrequencies()
print(sample_rates)

# extract the EEG data
eeg_data = []
for i in range(edf_file.signals_in_file):
    # only extract EEG channels (i.e. those with a sample rate of 100Hz)
    if sample_rates[i] == 1:
        eeg_data.append(edf_file.readSignal(i))

# convert the EEG data to a NumPy array
eeg_data = np.array(eeg_data)

print(eeg_data)


['ROC-LOC', 'LOC-ROC', 'F2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'F1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'C4-A1', 'EMG1-EMG2', 'ECG1-ECG2', 'TERMISTORE', 'TORACE', 'ADDOME', 'Dx1-DX2', 'SX1-SX2', 'Posizione', 'HR', 'SpO2']
[512 512 512 512 512 512 512 512 512 512 512 256 512  64  64  64 256 256
  64   1   1]
[[86.07919432 86.07919432 85.07820249 ... 73.08155947 73.08155947
  73.08155947]
 [95.01792935 95.01792935 95.01792935 ... 98.02090486 98.02090486
  98.02090486]]


In [ ]:
# get the number of samples in the file
n_samples = edf_file.getNSamples()[0]
print(n_samples)

# get the sample frequency
freq = edf_file.getSampleFrequency(0)
print(freq)

# get the data for the first channel
channel_data = edf_file.readSignal(0)
print(channel_data)

# create an array of all the time intervals for the channel
time_intervals = np.arange(n_samples) / freq
print(time_intervals)

17725440
512.0
[ 28.96970238  19.93384158   8.21164377 ... 244.94509158 242.44191392
 241.61769689]
[0.00000000e+00 1.95312500e-03 3.90625000e-03 ... 3.46199941e+04
 3.46199961e+04 3.46199980e+04]


In [ ]:
# close the EDF file
edf_file.close()

In [ ]:
## Ambil list directory di vector
eeg_vec_dir = tf.io.gfile.glob(str(vector_dir) +'*.edf')
print(eeg_vec_dir)
eeg_vec_dir = tf.random.shuffle(eeg_vec_dir)
print("files : ", len(eeg_vec_dir))

[]
files :  0


In [ ]:
def get_samples(edf_file, channel_name):
    # open the EDF file
    with pyedflib.EdfReader(edf_file) as edf:
        # get the index of the channel
        channel_index = edf.getSignalLabels().index(channel_name)
        
        # get all the data for the channel
        samples = edf.readSignal(channel_index)

    return samples

In [ ]:
sample = get_samples("/content/drive/MyDrive/dataset/eegcap/sdb/sdb2.edf","F2-F4")
print(sample.shape)

ValueError: ignored

In [ ]:
sample_dir = vector_dir
labels = ["sdb", "normal"]
channel_name = "F2-F4"
print(sample_dir, labels, channel_name)

/content/drive/MyDrive/dataset/eegcap ['sdb', 'normal'] F2-F4


In [ ]:
from tensorflow.python.ops.functional_ops import If
eeg_data = []
eeg_labels = []

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)

        with pyedflib.EdfReader(filepath) as edf:
          # get the index of the channel
          channel_index = edf.getSignalLabels().index(channel_name)
          
          # get all the data for the channel
          samples = edf.readSignal(channel_index)
        
        # with open(filepath, 'r') as file:
        #     recording = np.array(file.read().split(), dtype=np.float32)

        eeg_data.append(samples)

        if (label == 'normal'):
          label = 0
        elif (label == 'sdb'):
          label = 1

        eeg_labels.append(label)


eeg_data = np.array(eeg_data)
eeg_labels = np.array(eeg_labels)
print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

OSError: ignored

### CAP txt

#### window version

##### NON FFT

In [ ]:
eeg_sample = []
eeg_labell = []
max_len = 0  # variable to store maximum length of signals
window_size = 879936
stride = 0.1 # 10% overlap

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as file:
            print(filename)
            # skip the first line as it contains column headers
            file.readline()

            # read the remaining lines and extract signal values
            data = []
            for i, line in enumerate(file):
                values = line.strip().split(";")[4] # extract signal value from the 5th column
                data.append(float(values))

            # Pad data if shorter than window size
            if len(data) < window_size:
                padding = np.zeros(window_size - len(data))
                data = np.concatenate([data, padding])

            # Split data into segments
            stride_size = int(window_size * stride)
            for i in range(0, len(data) - window_size + 1, stride_size):
                segment = data[i:i+window_size]
                eeg_sample.append(segment)

                # if this is a new maximum length
                if len(segment) > max_len:
                    max_len = len(segment)

                if (label == 'normal'):
                    bi_label = 0
                elif (label == 'sdb'):
                    bi_label = 1

                eeg_labell.append(bi_label)


n1 c3p3.txt
sdb3 P4-O2 (2)
sdb3 P4-O2 (1)


##### FFT

In [ ]:
from scipy.fft import fft

eeg_sample = []
eeg_labell = []
max_len = 0  # variable to store maximum length of signals
window_size = 879936
stride = 0.1 # 10% overlap

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as file:
            print(filename)
            # skip the first line as it contains column headers
            file.readline()

            # read the remaining lines and extract signal values
            data = []
            for i, line in enumerate(file):
                values = line.strip().split(";")[4] # extract signal value from the 5th column
                data.append(float(values))

            # Pad data if shorter than window size
            if len(data) < window_size:
                padding = np.zeros(window_size - len(data))
                data = np.concatenate([data, padding])

            # Split data into segments
            stride_size = int(window_size * stride)
            for i in range(0, len(data) - window_size + 1, stride_size):
                segment = data[i:i+window_size]

                # Apply FFT to the segment
                fft_segment = fft(segment)

                eeg_sample.append(segment)

                # if this is a new maximum length
                if len(segment) > max_len:
                    max_len = len(segment)

                if (label == 'normal'):
                    bi_label = 0
                elif (label == 'sdb'):
                    bi_label = 1

                eeg_labell.append(bi_label)

n1 c3p3.txt
sdb3 P4-O2 (2)
sdb3 P4-O2 (1)


#### non window

In [ ]:
from tensorflow.python.ops.functional_ops import If

eeg_sample = []
eeg_labell = []
max_len = 0  # variable to store maximum length of signals

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as file:
            print(filename)
            # skip the first line as it contains column headers
            file.readline()

            # read the remaining lines and extract signal values
            data = []
            for i, line in enumerate(file):
                # # only extract signal values from every other line for "normal" data
                # if label == 'normal' and i % 2 == 0:
                #     values = line.strip().split(";")[4] # extract signal value from the 5th column
                #     data.append(float(values))
                # # extract signal values from every line for "sdb" data
                # elif label == 'sdb':
                #     values = line.strip().split(";")[4] # extract signal value from the 5th column
                #     data.append(float(values))

                values = line.strip().split(";")[4] # extract signal value from the 5th column
                data.append(float(values))
                
            # update max_len if necessary
            if len(data) > max_len:
                max_len = len(data)

            # convert the list of values to a numpy array
            data = np.array(data)

        eeg_sample.append(data)

        if (label == 'normal'):
            bi_label = 0
        elif (label == 'sdb'):
            bi_label = 1

        eeg_labell.append(bi_label)

n1 f2f4.txt
n1 f4c4.txt
n1 c3p3.txt
n1 p3o1.txt
n2 C4-A1.txt
sdb3 P4-O2 (4)
sdb3 P4-O2 (3)
sdb3 P4-O2 (5)
sdb3 P4-O2 (2)
sdb3 P4-O2 (8)
sdb3 P4-O2 (9)
sdb3 P4-O2 (7)
sdb3 P4-O2 (1)
sdb3 P4-O2 (6)


####check

In [ ]:
eeg_sample = np.array(eeg_sample)
eeg_labell = np.array(eeg_labell)
print("EEG data shape:", eeg_sample.shape)
print("EEG label shape:", eeg_labell.shape)

EEG data shape: (352, 879936)
EEG label shape: (352,)


In [ ]:
unique_labels, label_counts = np.unique(eeg_labell, return_counts=True)
print("Label counts:", dict(zip(unique_labels, label_counts)))

Label counts: {0: 192, 1: 160}


In [ ]:
eeg_data = eeg_sample
eeg_labels = eeg_labell

In [ ]:
print("EEG data shape:", eeg_data.shape)

for data in eeg_data:
  print( data.shape)

#### pad / truncate

In [ ]:
# pad or truncate all signals to the same length
eeg_data = pad_sequences(eeg_data, maxlen=7879936, dtype='float32', padding='post', truncating='post')

eeg_labels = np.array(eeg_labels)
print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

for data in eeg_data:
  print(data.shape)

EEG data shape: (18, 7879936)
EEG label shape: (18,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)
(7879936,)


### Worked

In [ ]:
from tensorflow.python.ops.functional_ops import If
eeg_data = []
eeg_labels = []

for label in labels:
    label_dir = os.path.join(sample_dir, label)
    for filename in os.listdir(label_dir):
        filepath = os.path.join(label_dir, filename)
        with open(filepath, 'r') as file:
            recording = np.array(file.read().split(), dtype=np.float32)

        eeg_data.append(recording)

        if (label == 'normal'):
          label = 0
        elif (label == 'sa'):
          label = 1

        eeg_labels.append(label)


eeg_data = np.array(eeg_data)
eeg_labels = np.array(eeg_labels)
print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

ValueError: ignored

###RNN

In [ ]:
# convert to 3D tensor
eeg_data = eeg_data.reshape((len(eeg_data), 1, -1))
eeg_labels = eeg_labels.reshape((len(eeg_labels), 1, -1))

print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

print(eeg_data)
print(eeg_labels)

EEG data shape: (2417, 1, 6000)
EEG label shape: (2417, 1, 1)
[[[ 1.92272472e+01  1.64804983e+00  1.70298481e+01 ...  2.38051629e+00
   -7.14154911e+00  3.47921634e+00]]

 [[-7.87401581e+00  3.11298299e+00 -6.40908241e+00 ...  4.94414949e+00
    5.31038284e+00  2.01428318e+00]]

 [[ 1.59311485e+01 -9.33894920e+00  1.22688150e+01 ...  1.08038816e+01
   -7.14154911e+00  1.70298481e+01]]

 ...

 [[-6.31752434e+01 -1.32393341e+02 -6.83025055e+01 ...  8.51492386e+01
    5.14557762e+01  7.41622391e+01]]

 [[-1.83116645e-01 -1.83116645e-01 -1.83116645e-01 ... -1.08221939e+02
   -1.55649147e+01 -9.86998749e+01]]

 [[-2.38051629e+00 -1.10419334e+02  2.17908802e+01 ...  2.04175064e+02
    4.55960464e+01  1.89159500e+02]]]
[[[1]]

 [[1]]

 [[1]]

 ...

 [[0]]

 [[0]]

 [[0]]]


### CNN

In [ ]:
eeg_data = np.array(eeg_data)

# check number of dimensions of eeg_data
print("EEG data dimensions:", eeg_data.ndim)

# swap second and third dimensions using swapaxes
# eeg_data = np.swapaxes(eeg_data, 1, 2)

# alternatively, you can reshape the array to match the desired axes
eeg_data = eeg_data.reshape(eeg_data.shape[0], eeg_data.shape[1], 1)
eeg_labels = eeg_labels.reshape(eeg_labels.shape[0], 1, 1)

# eeg_data = eeg_data.reshape(eeg_data.shape[0], eeg_data.shape[2], eeg_data.shape[1])

print("EEG data shape:", eeg_data.shape)
print("EEG label shape:", eeg_labels.shape)

EEG data dimensions: 2
EEG data shape: (352, 879936, 1)
EEG label shape: (352, 1, 1)


##SPLIT

###worked

In [ ]:
# Generate an array of indices for the samples
indices = np.arange(len(eeg_data))

# Shuffle the indices randomly
np.random.shuffle(indices)

# Use the shuffled indices to shuffle the data and labels arrays
eeg_data = eeg_data[indices]
eeg_labels = eeg_labels[indices]

In [ ]:
from collections import Counter


# Split the data into training, validation, and testing sets
num_samples = len(eeg_data)
num_train = int(num_samples * 0.7)
num_val = int(num_samples * 0.15)
num_test = num_samples - num_train - num_val

train_data = eeg_data[:num_train]
train_labels = eeg_labels[:num_train]

val_data = eeg_data[num_train:num_train+num_val]
val_labels = eeg_labels[num_train:num_train+num_val]

test_data = eeg_data[num_train+num_val:]
test_labels = eeg_labels[num_train+num_val:]

print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print("Number of testing samples:", len(test_data))

Number of training samples: 246
Number of validation samples: 52
Number of testing samples: 54


In [ ]:
unique_labels, label_counts = np.unique(train_labels, return_counts=True)
print("training samples:", (dict(zip(unique_labels, label_counts))))

unique_labels, label_counts = np.unique(val_labels, return_counts=True)
print("validation samples:",(dict(zip(unique_labels, label_counts))))

unique_labels, label_counts = np.unique(test_labels, return_counts=True)
print("testing samples:", (dict(zip(unique_labels, label_counts))))

training samples: {0: 141, 1: 105}
validation samples: {0: 26, 1: 26}
testing samples: {0: 25, 1: 29}


###idk

In [ ]:
import glob

# Define the directory where the labeled samples are stored
data_dir = "/content/drive/MyDrive/dataset/sample"

# Define the class labels
class_labels = ["normal", "sa"]

# Create a list of file paths for each class
class_file_paths = []
for label in class_labels:
    # Create a pattern to match the file names for this class
    pattern = f"{data_dir}/{label}/*.txt"
    # Get the file paths that match this pattern
    file_paths = glob.glob(pattern)
    # Add the file paths to the list for this class
    class_file_paths.append(file_paths)


In [ ]:
from sklearn.model_selection import train_test_split

# Define the percentage of data to use for training, validation, and test sets
train_percent = 0.6
val_percent = 0.2
test_percent = 0.2

# Create empty lists to hold the training, validation, and test sets
train_set = []
val_set = []
test_set = []

# Split the data for each class
for i, file_paths in enumerate(class_file_paths):
    # Split the file paths for this class into training and validation/test sets
    train_paths, val_test_paths = train_test_split(file_paths, test_size=val_percent+test_percent)
    # Split the validation/test file paths into validation and test sets
    val_paths, test_paths = train_test_split(val_test_paths, test_size=test_percent/(val_percent+test_percent))
    # Add the file paths to the appropriate set
    train_set.extend([(path, i) for path in train_paths])
    val_set.extend([(path, i) for path in val_paths])
    test_set.extend([(path, i) for path in test_paths])

# Shuffle the sets
random.shuffle(train_set)
random.shuffle(val_set)
random.shuffle(test_set)

# Print the size of each set
print(f"Number of training samples: {len(train_set)}")
print(f"Number of validation samples: {len(val_set)}")
print(f"Number of test samples: {len(test_set)}")


Number of training samples: 931
Number of validation samples: 310
Number of test samples: 312


# Modelling

## RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

rnn = Sequential()
rnn.add(SimpleRNN(64, input_shape=(879936,1)))
rnn.add(Dense(1, activation='sigmoid'))

rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 64)                4224      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 4,289
Trainable params: 4,289
Non-trainable params: 0
_________________________________________________________________


## CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

cnn = Sequential()
cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(879936, 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(16, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 879934, 32)        128       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 439967, 32)       0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 439965, 16)        1552      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 219982, 16)       0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3519712)           0         
                                                                 
 dense_2 (Dense)             (None, 16)               

# Execute Model

### RNN non FFT

In [ ]:
rnn.fit(train_data, train_labels, batch_size=16, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
16/16 [==============================] - 2277s 143s/step - loss: 0.7070 - accuracy: 0.5325 - val_loss: 0.7579 - val_accuracy: 0.4231
Epoch 2/10
16/16 [==============================] - 2298s 144s/step - loss: 0.6771 - accuracy: 0.6138 - val_loss: 0.6916 - val_accuracy: 0.5577
Epoch 3/10
16/16 [==============================] - 2607s 164s/step - loss: 0.6669 - accuracy: 0.5854 - val_loss: 0.7036 - val_accuracy: 0.5192
Epoch 4/10
16/16 [==============================] - 2412s 151s/step - loss: 0.6550 - accuracy: 0.6667 - val_loss: 0.6843 - val_accuracy: 0.5577
Epoch 5/10
16/16 [==============================] - 2280s 143s/step - loss: 0.6421 - accuracy: 0.6301 - val_loss: 0.6810 - val_accuracy: 0.5577
Epoch 6/10
16/16 [==============================] - 2312s 146s/step - loss: 0.6306 - accuracy: 0.6667 - val_loss: 0.6939 - val_accuracy: 0.5385
Epoch 7/10
16/16 [==============================] - 2632s 166s/step - loss: 0.6185 - accuracy: 0.6748 - val_loss: 0.6731 - val_accuracy:

In [ ]:
Ttest_loss, test_accuracy = rnn.evaluate(test_data, t est_labels)
print(f"Test loss: ", test_loss)
print(f"Test accuracy:" ,test_accuracy)

2/2 [==============================] - 86s 40s/step - loss: 0.6822 - accuracy: 0.5741
Test loss:  0.6821998953819275
Test accuracy: 0.5740740895271301


In [ ]:
test_data = np.random.rand(10, 1, 6000) # generate some random test data
# print(test_data)
predictions = rnn.predict(test_data)
print(predictions)

### RNN FFT

In [ ]:
rnn.fit(train_data, train_labels, batch_size=16, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
16/16 [==============================] - 2202s 138s/step - loss: 0.7136 - accuracy: 0.5854 - val_loss: 0.6998 - val_accuracy: 0.5000
Epoch 2/10
16/16 [==============================] - 2193s 138s/step - loss: 0.6780 - accuracy: 0.5569 - val_loss: 0.6873 - val_accuracy: 0.6346
Epoch 3/10
16/16 [==============================] - 2161s 136s/step - loss: 0.6753 - accuracy: 0.6098 - val_loss: 0.6845 - val_accuracy: 0.5385
Epoch 4/10
16/16 [==============================] - 2166s 136s/step - loss: 0.6651 - accuracy: 0.5935 - val_loss: 0.6717 - val_accuracy: 0.5192
Epoch 5/10
16/16 [==============================] - 2180s 137s/step - loss: 0.6453 - accuracy: 0.6626 - val_loss: 0.6676 - val_accuracy: 0.5192
Epoch 6/10
16/16 [==============================] - 2156s 135s/step - loss: 0.6368 - accuracy: 0.6423 - val_loss: 0.6494 - val_accuracy: 0.5385
Epoch 7/10
16/16 [==============================] - 2154s 135s/step - loss: 0.6214 - accuracy: 0.6748 - val_loss: 0.6620 - val_accuracy:

In [ ]:
test_loss, test_accuracy = rnn.evaluate(test_data, test_labels)
print(f"Test loss: ", test_loss)
print(f"Test accuracy:" ,test_accuracy)

2/2 [==============================] - 85s 39s/step - loss: 0.8156 - accuracy: 0.4259
Test loss:  0.8156340718269348
Test accuracy: 0.42592594027519226


In [ ]:
test_data = np.random.rand(10, 1, 6000) # generate some random test data
# print(test_data)
predictions = rnn.predict(test_data)
print(predictions)

###CNN Non FFt

In [ ]:
cnn.fit(train_data, train_labels, batch_size=16, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
16/16 [==============================] - 14s 509ms/step - loss: 163.9244 - accuracy: 0.5081 - val_loss: 0.6934 - val_accuracy: 0.4615
Epoch 2/10
16/16 [==============================] - 6s 407ms/step - loss: 0.6924 - accuracy: 0.5691 - val_loss: 0.6937 - val_accuracy: 0.4615
Epoch 3/10
16/16 [==============================] - 7s 410ms/step - loss: 0.6921 - accuracy: 0.5691 - val_loss: 0.6939 - val_accuracy: 0.4615
Epoch 4/10
16/16 [==============================] - 7s 410ms/step - loss: 0.6918 - accuracy: 0.5691 - val_loss: 0.6941 - val_accuracy: 0.4615
Epoch 5/10
16/16 [==============================] - 7s 415ms/step - loss: 0.6915 - accuracy: 0.5691 - val_loss: 0.6943 - val_accuracy: 0.4615
Epoch 6/10
16/16 [==============================] - 7s 414ms/step - loss: 0.6912 - accuracy: 0.5691 - val_loss: 0.6946 - val_accuracy: 0.4615
Epoch 7/10
16/16 [==============================] - 7s 414ms/step - loss: 0.6908 - accuracy: 0.5691 - val_loss: 0.6948 - val_accuracy: 0.4615
Epo

In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_data, test_labels)
print(f"Test loss: ", test_loss)
print(f"Test accuracy: ", test_accuracy)

2/2 [==============================] - 1s 230ms/step - loss: 0.6925 - accuracy: 0.5185
Test loss:  0.6925281882286072
Test accuracy:  0.5185185074806213


In [ ]:
test_data = np.random.rand(10, 6000, 1) # generate some random test data
# print(test_data)
predictions = cnn.predict(test_data)
print(predictions)

1/1 [==============================] - 0s 87ms/step
[[0.5672535 ]
 [0.5664299 ]
 [0.5770618 ]
 [0.56795883]
 [0.5806872 ]
 [0.5802547 ]
 [0.55819255]
 [0.57009065]
 [0.5702784 ]
 [0.5656687 ]]


###CNN FFt

In [ ]:
cnn.fit(train_data, train_labels, batch_size=16, epochs=10, validation_data=(val_data, val_labels))

Epoch 1/10
16/16 [==============================] - 141s 9s/step - loss: 44.6973 - accuracy: 0.5407 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/10
16/16 [==============================] - 139s 9s/step - loss: 0.6814 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/10
16/16 [==============================] - 140s 9s/step - loss: 0.6877 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/10
16/16 [==============================] - 137s 9s/step - loss: 0.6864 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/10
16/16 [==============================] - 155s 10s/step - loss: 0.6837 - accuracy: 0.5732 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
16/16 [==============================] - 141s 9s/step - loss: 0.6828 - accuracy: 0.5732 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 7/10
16/16 [==============================] - 139s 9s/step - loss: 0.6824 - accuracy: 0.5732 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 8/10


In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_data, test_labels)
print(f"Test loss: ", test_loss)
print(f"Test accuracy: ", test_accuracy)

2/2 [==============================] - 11s 4s/step - loss: 0.6987 - accuracy: 0.3889
Test loss:  0.698698878288269
Test accuracy:  0.3888888955116272


##CLEAN

### one time use

In [ ]:
del sample_dir 
del labels

In [ ]:
del max_len 
del window_size 
del stride 

del label_dir
del filename
del filepath 

del file

del data
del i
del line
del values
# del padding
del stride_size
del segment
del label
del bi_label

In [ ]:
del indices

In [ ]:
del num_samples
del num_train
del num_val
del num_test

In [ ]:
del unique_labels
del label_counts

In [ ]:
del test_loss
del test_accuracy

### can be use

In [ ]:
del eeg_sample 
del eeg_labell 

In [ ]:
del eeg_data
del eeg_labels

In [ ]:
del rnn

In [ ]:
del cnn